In [25]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta

# Define the ticker symbol for Apple
ticker = 'AAPL'

end_date = datetime.today()
start_date = end_date - timedelta(days=60)

try:
    # Fetch all hourly data at once
    data = yf.download(
        ticker,
        start=start_date,
        end=end_date,
        interval='5m',
        progress=False
    )
except Exception as e:
    print(f"Error fetching data: {e}")
    data = pd.DataFrame()

# Initialize counters
days_with_positive_first_hour = 0
days_positive_after_positive_first_hour = 0

if not data.empty:
    # Ensure datetime index
    data.index = pd.to_datetime(data.index)
    
    # Group data by date
    grouped = data.groupby(data.index.date)
    
    for date, group in grouped:
        if not group.empty:
            # Get first hour's data
            first_hour = group.iloc[0]
            # Check if first hour ended higher than it started
            if first_hour['Close'] > first_hour['Open']:
                days_with_positive_first_hour += 1
                day_close = group['Close'].iloc[-1]
                if day_close > first_hour['Close']:
                    days_positive_after_positive_first_hour += 1

# Calculate the percentage
if days_with_positive_first_hour > 0:
    percentage = (days_positive_after_positive_first_hour / days_with_positive_first_hour) * 100
    print(days_with_positive_first_hour, days_positive_after_positive_first_hour)
    print(f"Percentage of days ending positive given the first hour was positive: {percentage:.2f}%")
else:
    print("No days with positive first hour were found.")
    
# Note:
# Fetching hourly data for 1 year should be supported by yfinance.


21 14
Percentage of days ending positive given the first hour was positive: 66.67%


In [26]:
positive_days = 0
total_days = 0
for date, group in grouped:
    total_days += 1
    if not group.empty:
        day_open = group.iloc[0]['Open']
        day_close = group['Close'].iloc[-1]
        if day_close > day_open:
            positive_days += 1

print(f"Total number of days with Close > Open: {positive_days}")
print(f"Total number of days: {total_days}")
print(f"Percentage of days with Close > Open: {positive_days / total_days * 100:.2f}%")

Total number of days with Close > Open: 28
Total number of days: 41
Percentage of days with Close > Open: 68.29%
